# Notebook #3: Queries, Visualización y Análisis

- En este tercer notebook haremos consultas sobre nuestra base de datos y las usaremos para generar visualizaciones y el análisis de las mismas.

- Las visualizaciones de Ploty no se renderizan en Github. Para poder verlas es necesaria la descarga del Notebook.

- El primer paso será importar las librerías necesarias:

In [1]:
# Librerías para tratamiento de datos
import pandas as pd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los resultado3s
import numpy as np

# Trabajar con bases de datos SQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones.

# Librerías para graficar
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

/Users/davidfranco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Objetivo del Análisis 

- Antes de plantearnos las queries que pretendemos realizar, debemos recordar los objetivos del proyecto, así como la estructura de los datos.

    **1. Comparación de Precios entre Supermercados:** Determinar qué supermercados ofrecen los precios más bajos y cuáles son más caros para cada producto.

    **2. Análisis de la Evolución de Precios:** Estudiar cómo han cambiado los precios de los productos a lo largo del tiempo en distintos supermercados.

    **3. Detección de Anomalías:** Identificar subidas o bajadas de precios inusuales que podrían señalar prácticas abusivas o promociones.

    **4. Análisis de la Dispersión de Precios:** Evaluar la variabilidad de los precios de un mismo producto en diferentes supermercados.

    **5. Comparación de Precios Promedio:** Calcular y comparar los precios promedio de cada producto en diferentes supermercados.

#### Diagrama Entidad Relación

<img src="../images/Diagrama_ER.png" width="300">

### 1. Comparación de Precios entre Supermercados

- Determinar qué supermercados ofrecen los precios más bajos y cuáles son más caros para cada producto.

In [2]:
query1 = '''
SELECT 
    s.nombre AS supermercado,
    c.nombre AS categoria,
    MIN(h.precioeur) AS precio_minimo,
    MAX(h.precioeur) AS precio_maximo
FROM 
    historico h
INNER JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
INNER JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
GROUP BY 
    s.nombre, c.nombre
ORDER BY 
    c.nombre, precio_minimo;
'''

In [3]:
resultado1 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query1)
resultado1

,supermercado,categoria,precio_minimo,precio_maximo
0,Alcampo,Aceite-de-girasol,1.29,6.92
1,Hipercor,Aceite-de-girasol,1.4,6.91
2,Eroski,Aceite-de-girasol,1.45,7.28
3,Carrefour,Aceite-de-girasol,1.45,40.67
4,Mercadona,Aceite-de-girasol,1.45,6.91
5,Dia,Aceite-de-girasol,1.49,10.99
6,Eroski,Aceite-de-oliva,1.89,64.59
7,Hipercor,Aceite-de-oliva,2.09,154.32
8,Alcampo,Aceite-de-oliva,2.83,62.99
9,Carrefour,Aceite-de-oliva,2.85,63.5


In [4]:
# La función melt de Pandas permite "aplanar" la información, transformándola de un formato ancho a uno largo, que permite a Ploty graficar de manera adecuada.

df_melt = resultado1.melt(
    id_vars=["supermercado", "categoria"], 
    value_vars=["precio_minimo", "precio_maximo"], 
    var_name="tipo_precio", 
    value_name="precio"
)

fig1 = px.bar(
    df_melt, 
    x="supermercado", 
    y="precio", 
    color="categoria", 
    barmode="group",
    facet_col="tipo_precio",  # Crear una columna por tipo de precio (mínimo, máximo)
    labels={
        "supermercado": "Supermercado",
        "precio": "Precio (EUR)",
        "categoria": "Categoría",
        "tipo_precio": "Tipo de Precio"
    },
    title="Comparación de Precios entre Supermercados para cada Categoría de Producto"
)

fig1.update_layout(
    xaxis_title="Supermercado",
    yaxis_title="Precio en EUR",
)
fig1.update_yaxes(matches=None)

fig1.show()

### 2. Análisis de la Evolución de Precios

- Estudiar cómo han cambiado los precios de los productos a lo largo del tiempo en distintos supermercados.


In [5]:
query2 = '''
SELECT 
    h.fecha,
    s.nombre AS supermercado,
    c.nombre AS categoria,
    ROUND(AVG(h.precioeur),2) AS precio_promedio
FROM 
    historico h
INNER JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
INNER JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
GROUP BY 
    h.fecha, s.nombre, c.nombre
ORDER BY 
    c.nombre, s.nombre, h.fecha;
'''

In [6]:
resultado2 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query2)
resultado2

,fecha,supermercado,categoria,precio_promedio
0,2024-06-25,Alcampo,Aceite-de-girasol,2.65
1,2024-06-26,Alcampo,Aceite-de-girasol,2.65
2,2024-06-27,Alcampo,Aceite-de-girasol,2.74
3,2024-06-28,Alcampo,Aceite-de-girasol,2.74
4,2024-06-29,Alcampo,Aceite-de-girasol,2.74
...,...,...,...,...
1957,2024-10-23,Mercadona,Leche,3.70
1958,2024-10-24,Mercadona,Leche,3.70
1959,2024-10-25,Mercadona,Leche,3.70
1960,2024-10-26,Mercadona,Leche,3.70


In [7]:
# Lista de categorías
categorias = resultado2['categoria'].unique()

# Crear subplots: una fila por categoría
fig2 = make_subplots(
    rows=len(categorias), 
    cols=1, 
    shared_xaxes=True, 
    subplot_titles=categorias,
    vertical_spacing=0.1
)

# Añadir una gráfica de línea para cada categoría y supermercado
for i, categoria in enumerate(categorias, start=1):
    # Filtrar los datos para la categoría actual
    df_categoria = resultado2[resultado2['categoria'] == categoria]
    
    for supermercado in df_categoria['supermercado'].unique():
        df_super = df_categoria[df_categoria['supermercado'] == supermercado]
        
        fig2.add_trace(
            go.Scatter(
                x=df_super['fecha'], 
                y=df_super['precio_promedio'], 
                mode='lines',
                name=f"{supermercado} - {categoria}",
                legendgroup=supermercado,  # Agrupa leyendas por supermercado
                line=dict(width=2),
            ),
            row=i, col=1
        )

fig2.update_layout(
    height=300 * 3,  # Ajusta la altura para acomodar todas las subplots
    title="Evolución de precios promedio por categoría ",
    xaxis_title="Fecha",
    yaxis_title="Precio Promedio en EUR",
    showlegend=True
)

fig2.show()

### 3. Detección de Anomalías

- Identificar subidas o bajadas de precios inusuales que podrían señalar prácticas abusivas o promociones.

In [8]:
query3 = '''
SELECT 
    h.fecha,
    s.nombre AS supermercado,
    c.nombre AS categoria,
    h.precioeur AS precio,
    h.vareuros AS variacion_euros,
    h.varporcentaje AS variacion_porcentaje
FROM 
    historico h
INNER JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
INNER JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
WHERE 
    h.varporcentaje >= 30 
    OR h.varporcentaje <= -30
ORDER BY 
    c.nombre, s.nombre, h.fecha;
'''

In [9]:
resultado3 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query3)
resultado3

,fecha,supermercado,categoria,precio,variacion_euros,variacion_porcentaje
0,2024-06-27,Alcampo,Aceite-de-girasol,2.34,0.59,33.71
1,2024-07-19,Alcampo,Aceite-de-girasol,1.29,-0.9,-41.1
2,2024-08-01,Alcampo,Aceite-de-girasol,2.19,0.9,69.77
3,2024-09-12,Alcampo,Aceite-de-girasol,1.39,-0.8,-36.53
4,2024-09-26,Alcampo,Aceite-de-girasol,2.19,0.8,57.55
...,...,...,...,...,...,...
188,2024-09-26,Eroski,Leche,1.42,0.33,30.28
189,2024-10-02,Eroski,Leche,1.45,0.36,33.03
190,2024-10-10,Eroski,Leche,2.81,0.82,41.21
191,2024-07-16,Mercadona,Leche,3.38,-2.33,-40.81


In [10]:
resultado3["precio"] = resultado3["precio"].apply(float)
resultado3["variacion_euros"] = resultado3["variacion_euros"].apply(float)
resultado3["variacion_porcentaje"] = resultado3["variacion_porcentaje"].apply(float)

In [11]:
fig3 = px.area(
    resultado3, 
    x="fecha", 
    y="variacion_porcentaje", 
    color="supermercado", 
    line_group="categoria", 
    labels={
        "fecha": "Fecha",
        "variacion_porcentaje": "Variación (%)",
        "supermercado": "Supermercado",
        "categoria": "Categoría"
    },
    title="Variaciones Inusuales en Precios por Supermercado y Categoría"
)


fig3.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Variación (%)"
)


fig3.show()


### 4. Análisis de la Dispersión de Precios

- Evaluar la variabilidad de los precios de un mismo producto en diferentes supermercados.

In [12]:
query4 = '''
SELECT 
    c.nombre AS categoria,
    ROUND(STDDEV(h.precioeur),2) AS dispersion_precio
FROM 
    historico h
INNER JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
GROUP BY 
    c.nombre
ORDER BY 
    dispersion_precio DESC;
'''

In [13]:
resultado4 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query4)
resultado4

,categoria,dispersion_precio
0,Aceite-de-oliva,16.89
1,Aceite-de-girasol,5.59
2,Leche,2.83


In [14]:
query4_1 = '''
SELECT 
    h.fecha, 
    c.nombre AS categoria, 
    s.nombre AS supermercado, 
    avg(h.precioeur) AS precio
FROM 
    historico h
INNER JOIN 
    categorias c ON h.id_categoria = c.id_categoria
INNER JOIN 
    supermercados s ON h.id_supermercado = s.id_supermercado
GROUP BY
    h.fecha, c.nombre, s.nombre;
'''

In [15]:
resultado4_1 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query4_1)
resultado4_1

,fecha,categoria,supermercado,precio
0,2024-06-22,Aceite-de-oliva,Alcampo,34.6525000000000000
1,2024-06-22,Aceite-de-oliva,Carrefour,18.8162500000000000
2,2024-06-22,Aceite-de-oliva,Hipercor,7.8500000000000000
3,2024-06-22,Aceite-de-oliva,Mercadona,14.2500000000000000
4,2024-06-23,Aceite-de-oliva,Alcampo,34.6525000000000000
...,...,...,...,...
1957,2024-10-27,Leche,Carrefour,1.5761111111111111
1958,2024-10-27,Leche,Dia,5.2892307692307692
1959,2024-10-27,Leche,Eroski,1.5711235955056180
1960,2024-10-27,Leche,Hipercor,1.6403797468354430


In [16]:
# Crear un diccionario de colores para los supermercados
color_map = {
    'Mercadona': 'rgb(31, 119, 180)',
    'Carrefour': 'rgb(255, 127, 14)',
    'Dia': 'rgb(44, 160, 44)',
    'Eroski': 'rgb(214, 39, 40)',
    'Hipercor': 'rgb(148, 103, 189)',
    'Alcampo': 'rgb(140, 86, 75)'
}

categorias = resultado4_1['categoria'].unique()
fig4_1 = sp.make_subplots(rows=len(categorias), cols=1, subplot_titles=categorias, shared_xaxes=True)


for i, categoria in enumerate(categorias):
    df_filtered = resultado4_1[resultado4_1['categoria'] == categoria]
    for supermercado in df_filtered['supermercado'].unique():
        df_supermercado = df_filtered[df_filtered['supermercado'] == supermercado]
        fig4_1.add_trace(
            go.Scatter(
                x=df_supermercado['fecha'], 
                y=df_supermercado['precio'], 
                mode='markers', 
                name=supermercado,
                marker=dict(color=color_map[supermercado])
            ),
            row=i + 1, col=1
        )

fig4_1.update_layout(
    height=300 * 3,
    title_text="Evolución de Precios por Supermercado y por Categoría",
    xaxis_title="Fecha",
    yaxis_title="Precio (EUR)",
    showlegend=True
)

fig4_1.show()


### 5. Comparación de Precios Promedio

- Calcular y comparar los precios promedio de cada producto en diferentes supermercados.

In [17]:
query5 = '''
SELECT 
    s.nombre AS supermercado,
    c.nombre AS categoria,
    ROUND(AVG(h.precioeur),2) AS precio_promedio
FROM 
    historico h
INNER JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
INNER JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
GROUP BY 
    s.nombre, c.nombre
ORDER BY 
    c.nombre, precio_promedio;
'''

In [18]:
resultado5 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query5)
resultado5

,supermercado,categoria,precio_promedio
0,Alcampo,Aceite-de-girasol,2.61
1,Hipercor,Aceite-de-girasol,3.01
2,Eroski,Aceite-de-girasol,3.13
3,Mercadona,Aceite-de-girasol,4.12
4,Dia,Aceite-de-girasol,5.54
5,Carrefour,Aceite-de-girasol,8.25
6,Mercadona,Aceite-de-oliva,13.04
7,Eroski,Aceite-de-oliva,13.25
8,Dia,Aceite-de-oliva,14.71
9,Carrefour,Aceite-de-oliva,17.31


In [19]:
resultado5["precio_promedio"] = resultado5["precio_promedio"].apply(float)

In [20]:
categorias = resultado5['categoria'].unique()
fig5 = sp.make_subplots(rows=len(categorias), cols=1, subplot_titles=categorias, shared_yaxes=False)

for i, categoria in enumerate(categorias):
    df_filtered = resultado5[resultado5['categoria'] == categoria]
    fig5.add_trace(
        go.Bar(x=df_filtered['supermercado'], y=df_filtered['precio_promedio'], name=categoria, text=df_filtered['precio_promedio'], textposition='auto'),
        row=i + 1, col=1
    )

fig5.update_layout(
    barmode='stack',  # Configurar las barras apiladas
    height=300 * 3,
    title_text="Precio Medio de Cada Categoría en Cada Supermercado",
    showlegend=False
)

fig5.show()


In [21]:
query6 = '''
SELECT 
    s.nombre AS supermercado, 
    c.nombre AS categoria, 
    COUNT(h.id_producto) AS total_productos
FROM 
    historico h
INNER JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
INNER JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
GROUP BY 
    s.nombre, c.nombre
ORDER BY 
    s.nombre, c.nombre;

'''

### 6. Cantidad de Productos por Supermercado

- Identificar la cantidad de referencias analizadas por supermercado.

In [22]:
resultado6 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query6)
resultado6

,supermercado,categoria,total_productos
0,Alcampo,Aceite-de-girasol,1176
1,Alcampo,Aceite-de-oliva,8933
2,Alcampo,Leche,13882
3,Carrefour,Aceite-de-girasol,1128
4,Carrefour,Aceite-de-oliva,8916
5,Carrefour,Leche,4559
6,Dia,Aceite-de-girasol,600
7,Dia,Aceite-de-oliva,2306
8,Dia,Leche,5434
9,Eroski,Aceite-de-girasol,848


In [23]:
import plotly.express as px

fig6 = px.bar(
    resultado6,
    x="supermercado",
    y="total_productos",
    color="categoria",
    labels={
        "supermercado": "Supermercado",
        "total_productos": "Total de Productos",
        "categoria": "Categoría"
    },
    title="Cantidad Total de Productos por Categoría y Supermercado",
    text="total_productos"  # Añadir las etiquetas de texto
)

# Actualizar el diseño de las etiquetas
fig6.update_traces(texttemplate='%{text}', textposition='outside')

fig6.update_layout(
    xaxis_title="Supermercado",
    yaxis_title="Total de Productos",
    barmode='stack'
)

fig6.show()
